In [1]:
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
import cell2location
import scvi
import torch
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.backends.backend_pdf as mpdf
import warnings
sys.path.append("/home/biolab/Projects/LAMs_wd/scripts")
warnings.filterwarnings('ignore')
data_type = 'float32'
from visium_qc_and_visualisation import read_and_qc
from config import lung_config

/home/ozlemtuna/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ozlemtuna/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/ozlemtuna/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


In [2]:
o_sig_matrix = pd.read_csv("/home/biolab/Projects/LAMs_wd/data/signature_matrix_sq_24_cell_type_renamed.csv", index_col=0)
o_sig_matrix

,Stromal,Endothelial_cell,Alveolar_cell_type_1,Alveolar_cell_type_2,transitional_club/AT2,other,Macrophage_alveolar,NK_cell,Mast_cell,Club,...,T_cell_CD8,Tumor_cells,T_cell_CD4,Plasma_cell,cDC1,cDC2,B_cell,Neutrophils,DC_mature,T_cell_regulatory
CDH11,429.229015,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000
DOCK9,1.000000,473.570096,1.000000,59.077085,1.000000,1.000000,1.0,1.000000,1.000000,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000
CLIC3,1.000000,1.000000,997.781399,212.359701,237.847267,1.000000,1.0,383.494818,1.000000,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000
CCNG2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,102.533423,1.0,...,1.000000,1.000000,1.000000,1.000000,52.746868,56.873905,1.000000,1.0,314.848934,1.000000
PCDH17,1.000000,560.654133,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CDKN3,1.000000,1.000000,1.000000,1.000000,1.000000,262.356384,1.0,1.000000,1.000000,1.0,...,1.000000,93.431375,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000
CAMK4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,...,175.347016,1.000000,213.361118,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,106.219895
WIF1,1.000000,1.000000,1.000000,209.595993,1.000000,1.000000,1.0,1.000000,1.000000,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000
TMEM156,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,...,1.000000,1.000000,76.778009,119.800078,1.000000,1.000000,167.259363,1.0,1.000000,1.000000


Run Cell2location

In [3]:
def run_cell2loc(ifolder, environment):
    seed = 42
    torch.manual_seed(seed)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    outDir = f"/home/biolab/Projects/LAMs_wd/results/cell2location/spatial_model_SC_Y_sig_matrix_adeno_LUSC_patients/"
    regDir = f"/home/biolab/Projects/LAMs_wd/data/E-MTAB-13530/{ifolder}/cell2location/regression_model/"
    import os
    os.makedirs(outDir + environment, exist_ok=True)

    # Pick up the scRNA-seq gene expression model
#     adata_sc_model = sc.read_h5ad(f"{regDir}{environment}/sc.h5ad")
#     adata_sc_model = sc.read_h5ad("/media/mllab/SSD/ST/lung-master/SpatialTranscriptomics/cell2location/regression_model/Tumour/sc.h5ad")
    
#     mod = cell2location.models.RegressionModel.load(f"{regDir}{environment}", adata_sc_model)
#     adata_sc_model = mod.export_posterior(adata_sc_model, sample_kwargs={'num_samples': 1000, 'batch_size': 2500})
#     adata_sc_model.write(f"{regDir}{environment}/sc.h5ad")

    inf_aver = None
    # Export estimated expression signature in each cluster
    
    inf_aver = o_sig_matrix.copy()

    # Scale up by average sample scaling factor. This corrects for sequencing depth
    # inf_aver = inf_aver * adata_sc_model.uns["mod"]["post_sample_means"]["sample_scaling"].mean() # Need to check if this is still necessary

    # Pick up Visium
    sample_names = lung_config[environment]["spatial"]["LUSC"]
    print(sample_names)
    #inDir = "/lustre/scratch118/opentargets/opentargets/OTAR2060/nelson/data/lung/visium/samples/{}/".format(environment.lower())
    inDir = "/home/biolab/Projects/LAMs_wd/data/E-TAMB-13530/"
    slides = [read_and_qc(sample_name, path=inDir) for sample_name in sample_names]
    adata_vis = slides[0].concatenate(slides[1:], batch_key="sample", uns_merge="unique", batch_categories=sample_names, index_unique=None)
    # Mitochondria-encoded (MT) genes should be removed for spatial mapping
    # Find mitochondria-encoded (MT) genes
    adata_vis.var["MT_gene"] = [gene.startswith("MT-") for gene in adata_vis.var["SYMBOL"]]
    # Remove MT genes for spatial mapping (keeping their counts in the object)
    adata_vis.obsm["MT"] = adata_vis[:, adata_vis.var["MT_gene"].values].X.toarray()
    adata_vis = adata_vis[:, ~adata_vis.var["MT_gene"].values]
    # Reset the index for compatibility with the scRNA-seq counts
    adata_vis.var.set_index("SYMBOL", inplace=True)

    # Find shared genes and subset both anndata and reference signatures
    adata_vis.var_names_make_unique()
    
    inf_aver_index = np.array(inf_aver.index, dtype=str)
    adata_vis_var_names = np.array(adata_vis.var_names, dtype=str)
    intersect = np.intersect1d(adata_vis_var_names, inf_aver_index)
    #intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)
    #adata_vis = adata_vis[:, intersect] # This doesn't work -- think there's some incompatibility between ScanPy and AnnData versions. Use the line below instead
    adata_vis = adata_vis[:, adata_vis.var.index.isin(intersect)].copy()
    inf_aver = inf_aver.loc[intersect, :]
    # Reindex for cell2location
    inf_aver = inf_aver.reindex(adata_vis.var.index)

    # Save on memory
#     del adata_sc_model
#     del mod

    # Training cell2location
    cell2location.models.Cell2location.setup_anndata(adata=adata_vis, batch_key="sample")
    mod = cell2location.models.Cell2location(adata_vis,
                                             cell_state_df=inf_aver,
                                             detection_alpha=200, # Controls the normalisation of the within-experiment variation of the RNA detection (default value for now)
                                             N_cells_per_location=8
                                             )
    '''
                                             **model_kwargs={
                                                # Prior on the number of cells, cell types and co-located groups. Hyperparameter inputs go here
                                                "cell_number_prior": {
                                                    # - N - the expected number of cells per location:
                                                    "cells_per_spot": 8,
                                                    # - A - the expected number of cell types per location:
                                                    "factors_per_spot": 9,
                                                    # - Y - the expected number of co-located cell type groups per location
                                                    "combs_per_spot": 5
                                                },
                                                # Prior beliefs on the sensitivity of spatial technology:
                                                "gene_level_prior": {
                                                    # Prior on the mean
                                                    "mean": 0.5,
                                                    # Prior on standard deviation
                                                    # A good choice of this value should be at least 2 times lower that the mean
                                                    "sd": 0.15
                                                }
                                             }
                                             )
    '''

    # Try 2500 batches initially, rather than training on the entire data-set
    mod.train(batch_size=2500, train_size=1, max_epochs=3000)
    mod.save(f"{outDir}{environment}", overwrite=True)
    adata_vis = mod.export_posterior(adata_vis, sample_kwargs={'num_samples': 1000, 'batch_size': 2500})
    adata_vis.write(f"{outDir}{environment}/sp_our_sig_matrix_all_cts_{environment}.h5ad")

    train_performance = mod.history["elbo_train"]
    train_performance.plot(logy=True)
    plt.xlabel("Training epochs")
    plt.ylabel("-ELBO loss")
    plt.title(f"Lung ({environment})")
    plt.legend(loc="best")
    plt.tight_layout()
    plt.savefig(f"{outDir}/ELBO_Cell2location_{environment}.png")
    plt.close()
    

In [4]:
run_cell2loc("dataset", "Tumour")

['spaceranger110_count_36210_OTAR_LNGsp9476045_GRCh38-2020-A', 'spaceranger110_count_38262_OTAR_LNGsp10206168_GRCh38-2020-A', 'spaceranger110_count_38262_OTAR_LNGsp10206167_GRCh38-2020-A', 'spaceranger110_count_36210_OTAR_LNGsp9476042_GRCh38-2020-A', 'spaceranger110_count_36210_OTAR_LNGsp9476043_GRCh38-2020-A']


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A4000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 3000/3000: 100%|██████████| 3000/3000 [16:00<00:00,  3.04it/s, v_num=1, elbo_train=4.38e+6]

`Trainer.fit` stopped: `max_epochs=3000` reached.


Sampling global variables, sample: 100%|██████████| 999/999 [00:13<00:00, 72.47it/s]
